In [212]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [213]:
hourly_df = pd.read_csv('NOAA_Hourly_Weather_Data_by_Station__Virginia_.csv')

/var/folders/4w/ryd_r6790xd370fr_0j0l_800000gn/T/ipykernel_79408/3264958441.py:1: DtypeWarning: Columns (0,2,7,9,10,12,13,14,16,17,19,20,22,23,25,26,28,29,31,32,34,35,37,40,42,43,44,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  hourly_df = pd.read_csv('NOAA_Hourly_Weather_Data_by_Station__Virginia_.csv')


In [214]:
hourly_df = hourly_df.drop(columns=["ElevationMeters", "ReportType", "Source", "SkyConditions", "SeaLevelPressureNumber", "SeaLevelPressure", "SeaLevelPressureIndicator", "PrecipitationIndicator", "AltimeterSetting", "RowIdentifier", "GeocodedLocation", "AltimeterSettingIndicator", "AltimeterSettingNumber", "Precipitation", "PressureChange", "PressureTendency", "StationPressureIndicator", "PresentWeatherType", "Visibility", "VisibilityIndicator", "VisibilityNumber", "DryBulbTemperatureIndicator", "DryBulbTemperature", "WetBulbTemperatureIndicator", "WindSpeedIndicator", "WindGustSpeed", "WindGustSpeedIndicator", "WindGustSpeedNumber", "WetBulbTemperature", "WindSpeed", "StationPressure", "WindDirection", "WindDirectionNumber", "WindDirectionIndicator", "RelativeHumidityIndicator", "RelativeHumidity", "DewPointTemperatureIndicator", "DewPointTemperature", "PrecipitationNumber", "Station", "StationName"])

In [215]:
hourly_df = hourly_df.rename(columns={"DryBulbTemperatureNumber": "DryBulb", "DewPointTemperatureNumber": "DewPoint", "RelativeHumidityNumber": "Humidity", "WindSpeedNumber": "WindSpeed", "StationPressureNumber": "Pressure", "WetBulbTemperatureNumber": "WetBulb"})

In [216]:
print(hourly_df.dtypes)

ICAO          object
Latitude     float64
Longitude    float64
DateTime      object
DryBulb      float64
WetBulb      float64
DewPoint     float64
Humidity     float64
WindSpeed    float64
Pressure     float64
dtype: object


In [217]:
hourly_df["DateTime"] = pd.to_datetime(hourly_df["DateTime"], format="%m/%d/%Y %H:%M:%S %p")

In [218]:
grouped = hourly_df.groupby([hourly_df['ICAO'], hourly_df['DateTime'].dt.date])

aggregated = grouped.agg({
    'DryBulb': ['max', 'min'],
    'WetBulb': ['max', 'min'],
    'Humidity': 'mean',
    'WindSpeed': 'mean',
    'Pressure': 'mean',
    'Latitude': 'mean',
    'Longitude': 'mean'
}).reset_index()

aggregated.columns = ['_'.join(col).strip() for col in aggregated.columns.values]

aggregated = aggregated.rename(columns={
    'ICAO_': 'ICAO',
    'DateTime_': 'DateTime',
    'DryBulb_max': 'High',
    'DryBulb_min': 'Low',
    'WetBulb_max': 'WetBulb_High',
    'WetBulb_min': 'WetBulb_Low',
    'Humidity_mean': 'Humidity',
    'WindSpeed_mean': 'WindSpeed',
    'Pressure_mean': 'Pressure',
    'Latitude_mean': 'Latitude',
    'Longitude_mean': 'Longitude'
})

In [219]:
print(aggregated.head())

   ICAO    DateTime  High   Low  WetBulb_High  WetBulb_Low   Humidity  \
0  K0VG  2017-01-01  48.0  37.0          48.0         37.0  98.463768   
1  K0VG  2017-01-02  63.0  46.0          58.0         46.0  93.873239   
2  K0VG  2017-01-03  59.0  54.0          58.0         52.0  96.434783   
3  K0VG  2017-01-04  54.0  25.0          52.0         24.0  71.333333   
4  K0VG  2017-01-05  36.0  23.0          31.0         22.0  76.521739   

   WindSpeed   Pressure  Latitude  Longitude  
0   0.253521  28.616479    36.654    -83.218  
1   0.902778  28.598889    36.654    -83.218  
2   4.126761  28.294789    36.654    -83.218  
3   6.671429  28.443571    36.654    -83.218  
4   0.828571  28.469286    36.654    -83.218  


In [220]:
High_q_low = aggregated["High"].quantile(0.05)
High_q_hi  = aggregated["High"].quantile(0.95)

Low_q_low = aggregated["Low"].quantile(0.05)
Low_q_hi  = aggregated["Low"].quantile(0.95)

WetBulb_High_q_low = aggregated["WetBulb_High"].quantile(0.05)
WetBulb_High_q_hi  = aggregated["WetBulb_High"].quantile(0.95)

WetBulb_Low_q_low = aggregated["WetBulb_Low"].quantile(0.05)
WetBulb_Low_q_hi  = aggregated["WetBulb_Low"].quantile(0.95)

Humidity_q_low = aggregated["Humidity"].quantile(0.05)
Humidity_q_hi  = aggregated["Humidity"].quantile(0.95)

WindSpeed_q_low = aggregated["WindSpeed"].quantile(0.05)
WindSpeed_q_hi  = aggregated["WindSpeed"].quantile(0.95)

Pressure_q_low = aggregated["Pressure"].quantile(0.05)
Pressure_q_hi  = aggregated["Pressure"].quantile(0.95)

aggregated = aggregated[(aggregated["High"] < High_q_hi) & (aggregated["High"] > High_q_low)]
aggregated = aggregated[(aggregated["Low"] < Low_q_hi) & (aggregated["Low"] > Low_q_low)]
aggregated = aggregated[(aggregated["WetBulb_High"] < WetBulb_High_q_hi) & (aggregated["WetBulb_High"] > WetBulb_High_q_low)]
aggregated = aggregated[(aggregated["WetBulb_Low"] < WetBulb_Low_q_hi) & (aggregated["WetBulb_Low"] > WetBulb_Low_q_low)]
aggregated = aggregated[(aggregated["Humidity"] < Humidity_q_hi) & (aggregated["Humidity"] > Humidity_q_low)]
aggregated = aggregated[(aggregated["Pressure"] < Pressure_q_hi) & (aggregated["Pressure"] > Pressure_q_low)]

aggregated["DateTime"] = pd.to_datetime(aggregated["DateTime"], format="%Y-%m-%d")

aggregated['year'] = aggregated['DateTime'].dt.year
aggregated['month'] = aggregated['DateTime'].dt.month
aggregated['day'] = aggregated['DateTime'].dt.day

# Calculate the day of the year for each date
aggregated['day_of_year'] = aggregated.apply(lambda row: (pd.Timestamp(year=row['year'], month=row['month'], day=row['day']) - pd.Timestamp(year=row['year'], month=1, day=1)).days, axis=1)
aggregated = aggregated.drop(columns=['year', 'month', 'day'])

def min_max_normalize(df, column_name):
    min_value = df[column_name].min()
    max_value = df[column_name].max()

    # Perform min-max normalization and create a new column with the results
    df[column_name] = (df[column_name] - min_value) / (max_value - min_value)

min_max_normalize(aggregated, 'High')
min_max_normalize(aggregated, 'Low')
min_max_normalize(aggregated, 'WetBulb_High')
min_max_normalize(aggregated, 'WetBulb_Low')
min_max_normalize(aggregated, 'Humidity')
min_max_normalize(aggregated, 'WindSpeed')
min_max_normalize(aggregated, 'Pressure')
min_max_normalize(aggregated, 'Latitude')
min_max_normalize(aggregated, 'Longitude')
min_max_normalize(aggregated, 'day_of_year')

print(aggregated.head())

    ICAO   DateTime  High       Low  WetBulb_High  WetBulb_Low  Humidity  \
1   K0VG 2017-01-02  0.46  0.468085      0.558140     0.510204  0.986957   
3   K0VG 2017-01-04  0.28  0.021277      0.418605     0.061224  0.525714   
9   K0VG 2017-01-10  0.10  0.085106      0.162791     0.081633  0.395260   
11  K0VG 2017-01-12  0.46  0.553191      0.558140     0.591837  0.819022   
14  K0VG 2017-01-15  0.34  0.553191      0.511628     0.591837  0.965677   

    WindSpeed  Pressure  Latitude     Longitude  day_of_year  
1    0.025270  0.436346  0.031518  1.833186e-15     0.002740  
3    0.186746  0.383248  0.031518  1.833186e-15     0.008219  
9    0.013996  0.497882  0.031518  1.833186e-15     0.024658  
11   0.163083  0.489889  0.031518  1.833186e-15     0.030137  
14   0.031880  0.508566  0.031518  1.833186e-15     0.038356  


In [221]:
with_previous = aggregated
with_previous['High-1'] = aggregated.shift(1)['High']
with_previous['High-2'] = aggregated.shift(2)['High']
with_previous['High-3'] = aggregated.shift(3)['High']
with_previous['High-4'] = aggregated.shift(4)['High']
with_previous['Low-1'] = aggregated.shift(1)['Low']
with_previous['Low-2'] = aggregated.shift(2)['Low']
with_previous['Low-3'] = aggregated.shift(3)['Low']
with_previous['Low-4'] = aggregated.shift(4)['Low']
with_previous['WetBulb_High-1'] = aggregated.shift(1)['WetBulb_High']
with_previous['WetBulb_High-2'] = aggregated.shift(2)['WetBulb_High']
with_previous['WetBulb_High-3'] = aggregated.shift(3)['WetBulb_High']
with_previous['WetBulb_High-4'] = aggregated.shift(4)['WetBulb_High']
with_previous['WetBulb_Low-1'] = aggregated.shift(1)['WetBulb_Low']
with_previous['WetBulb_Low-2'] = aggregated.shift(2)['WetBulb_Low']
with_previous['WetBulb_Low-3'] = aggregated.shift(3)['WetBulb_Low']
with_previous['WetBulb_Low-4'] = aggregated.shift(4)['WetBulb_Low']
with_previous['Humidity-1'] = aggregated.shift(1)['Humidity']
with_previous['Humidity-2'] = aggregated.shift(2)['Humidity']
with_previous['Humidity-3'] = aggregated.shift(3)['Humidity']
with_previous['Humidity-4'] = aggregated.shift(4)['Humidity']
with_previous['WindSpeed-1'] = aggregated.shift(1)['WindSpeed']
with_previous['WindSpeed-2'] = aggregated.shift(2)['WindSpeed']
with_previous['WindSpeed-3'] = aggregated.shift(3)['WindSpeed']
with_previous['WindSpeed-4'] = aggregated.shift(4)['WindSpeed']
with_previous['Pressure-1'] = aggregated.shift(1)['Pressure']
with_previous['Pressure-2'] = aggregated.shift(2)['Pressure']
with_previous['Pressure-3'] = aggregated.shift(3)['Pressure']
with_previous['Pressure-4'] = aggregated.shift(4)['Pressure']

with_previous = with_previous.dropna()

print(with_previous)

        ICAO   DateTime  High       Low  WetBulb_High  WetBulb_Low  Humidity  \
14      K0VG 2017-01-15  0.34  0.553191      0.511628     0.591837  0.965677   
15      K0VG 2017-01-16  0.48  0.553191      0.558140     0.591837  0.921339   
17      K0VG 2017-01-18  0.38  0.255319      0.511628     0.306122  0.637062   
18      K0VG 2017-01-19  0.30  0.255319      0.348837     0.306122  0.905992   
19      K0VG 2017-01-20  0.52  0.468085      0.558140     0.510204  0.848324   
...      ...        ...   ...       ...           ...          ...       ...   
117663  KXSA 2023-02-28  0.42  0.361702      0.465116     0.428571  0.914802   
117664  KXSA 2023-03-01  0.34  0.170213      0.418605     0.224490  0.880697   
117665  KXSA 2023-03-02  0.52  0.446809      0.627907     0.489796  0.878464   
117667  KXSA 2023-03-04  0.48  0.297872      0.697674     0.306122  0.305448   
117668  KXSA 2023-03-05  0.48  0.106383      0.023256     0.265306  0.506956   

        WindSpeed  Pressure  Latitude  

In [222]:
print(with_previous.dtypes)

ICAO                      object
DateTime          datetime64[ns]
High                     float64
Low                      float64
WetBulb_High             float64
WetBulb_Low              float64
Humidity                 float64
WindSpeed                float64
Pressure                 float64
Latitude                 float64
Longitude                float64
day_of_year              float64
High-1                   float64
High-2                   float64
High-3                   float64
High-4                   float64
Low-1                    float64
Low-2                    float64
Low-3                    float64
Low-4                    float64
WetBulb_High-1           float64
WetBulb_High-2           float64
WetBulb_High-3           float64
WetBulb_High-4           float64
WetBulb_Low-1            float64
WetBulb_Low-2            float64
WetBulb_Low-3            float64
WetBulb_Low-4            float64
Humidity-1               float64
Humidity-2               float64
Humidity-3

In [223]:
print(with_previous.head())

    ICAO   DateTime  High       Low  WetBulb_High  WetBulb_Low  Humidity  \
14  K0VG 2017-01-15  0.34  0.553191      0.511628     0.591837  0.965677   
15  K0VG 2017-01-16  0.48  0.553191      0.558140     0.591837  0.921339   
17  K0VG 2017-01-18  0.38  0.255319      0.511628     0.306122  0.637062   
18  K0VG 2017-01-19  0.30  0.255319      0.348837     0.306122  0.905992   
19  K0VG 2017-01-20  0.52  0.468085      0.558140     0.510204  0.848324   

    WindSpeed  Pressure  Latitude  ...  Humidity-3  Humidity-4  WindSpeed-1  \
14   0.031880  0.508566  0.031518  ...    0.525714    0.986957     0.163083   
15   0.011997  0.493183  0.031518  ...    0.395260    0.525714     0.031880   
17   0.132184  0.448596  0.031518  ...    0.819022    0.395260     0.011997   
18   0.004665  0.412985  0.031518  ...    0.965677    0.819022     0.132184   
19   0.062686  0.348129  0.031518  ...    0.921339    0.965677     0.004665   

    WindSpeed-2  WindSpeed-3  WindSpeed-4  Pressure-1  Pressure-2  P

In [224]:
with_previous.corr(numeric_only=True).style.background_gradient(cmap='coolwarm')

,High,Low,WetBulb_High,WetBulb_Low,Humidity,WindSpeed,Pressure,Latitude,Longitude,day_of_year,High-1,High-2,High-3,High-4,Low-1,Low-2,Low-3,Low-4,WetBulb_High-1,WetBulb_High-2,WetBulb_High-3,WetBulb_High-4,WetBulb_Low-1,WetBulb_Low-2,WetBulb_Low-3,WetBulb_Low-4,Humidity-1,Humidity-2,Humidity-3,Humidity-4,WindSpeed-1,WindSpeed-2,WindSpeed-3,WindSpeed-4,Pressure-1,Pressure-2,Pressure-3,Pressure-4
High,1.000000,0.847015,0.937418,0.856090,0.107011,-0.176875,-0.036222,-0.039013,-0.033303,0.120640,0.814042,0.720927,0.681314,0.666290,0.733468,0.669809,0.656094,0.653369,0.764264,0.678145,0.650634,0.640643,0.755276,0.685066,0.667741,0.664493,0.120006,0.092119,0.096984,0.107057,-0.254649,-0.222601,-0.192886,-0.170979,-0.002300,-0.005230,-0.014339,-0.018591
Low,0.847015,1.000000,0.908672,0.986957,0.304495,-0.044339,-0.036146,-0.035150,0.009329,0.131198,0.809302,0.742288,0.707539,0.687158,0.847709,0.754688,0.726209,0.711217,0.824237,0.727562,0.690582,0.674871,0.858974,0.764041,0.731359,0.715857,0.233931,0.135791,0.106926,0.112943,-0.163209,-0.165819,-0.137859,-0.118370,0.006369,0.015515,0.005046,-0.002025
WetBulb_High,0.937418,0.908672,1.000000,0.916979,0.354906,-0.112339,-0.030082,-0.052060,0.000670,0.139138,0.811642,0.721411,0.682357,0.667298,0.782461,0.693629,0.670232,0.665176,0.807431,0.697609,0.665007,0.654160,0.809602,0.711258,0.683492,0.678296,0.243700,0.141467,0.127258,0.136086,-0.248950,-0.218803,-0.180743,-0.155029,0.025660,0.025973,0.012448,0.005261
WetBulb_Low,0.856090,0.986957,0.916979,1.000000,0.369202,-0.116100,-0.041069,-0.040484,-0.005681,0.153576,0.809775,0.747101,0.714555,0.696080,0.843848,0.756015,0.729259,0.715882,0.827750,0.736272,0.700417,0.686008,0.863385,0.771396,0.740323,0.725636,0.268364,0.165071,0.133944,0.136402,-0.202259,-0.193826,-0.163772,-0.141650,-0.000796,0.006008,-0.004176,-0.011299
Humidity,0.107011,0.304495,0.354906,0.369202,1.000000,-0.204653,-0.030904,-0.068816,-0.005941,0.157418,0.131534,0.152333,0.157848,0.156545,0.218764,0.179649,0.167267,0.161286,0.233420,0.196169,0.189064,0.184654,0.265005,0.205340,0.190464,0.183611,0.456527,0.231941,0.185102,0.174912,-0.225771,-0.140863,-0.104407,-0.086787,0.038454,0.027714,0.014938,0.007881
WindSpeed,-0.176875,-0.044339,-0.112339,-0.116100,-0.204653,1.000000,0.113929,0.015404,0.251273,-0.193835,-0.145399,-0.169791,-0.167102,-0.162529,-0.064919,-0.109948,-0.113976,-0.115069,-0.103718,-0.157621,-0.155987,-0.150725,-0.096751,-0.137092,-0.142293,-0.139742,-0.050814,-0.106262,-0.117032,-0.103923,0.479016,0.359087,0.360034,0.350292,0.149065,0.170561,0.167379,0.167537
Pressure,-0.036222,-0.036146,-0.030082,-0.041069,-0.030904,0.113929,1.000000,0.188060,0.879275,-0.008993,-0.030353,-0.022521,-0.020978,-0.020835,-0.025968,-0.010201,-0.006467,-0.004624,-0.019243,-0.002190,0.000589,0.001562,-0.037740,-0.020810,-0.016056,-0.015066,-0.041003,-0.011275,-0.001971,-0.000836,0.168950,0.175768,0.174652,0.173333,0.973194,0.957816,0.954007,0.952225
Latitude,-0.039013,-0.035150,-0.052060,-0.040484,-0.068816,0.015404,0.188060,1.000000,0.189100,0.023545,-0.038936,-0.038421,-0.038264,-0.038148,-0.035075,-0.034853,-0.034701,-0.034443,-0.052152,-0.051633,-0.051500,-0.051309,-0.040362,-0.040199,-0.040023,-0.039770,-0.068968,-0.069561,-0.069463,-0.069213,0.015021,0.015278,0.015456,0.015441,0.188032,0.187891,0.187787,0.187612
Longitude,-0.033303,0.009329,0.000670,-0.005681,-0.005941,0.251273,0.879275,0.189100,1.000000,-0.033962,-0.033352,-0.033352,-0.033311,-0.033190,0.009304,0.009409,0.009577,0.009747,0.000534,0.000488,0.000414,0.000496,-0.005752,-0.005658,-0.005506,-0.005423,-0.006102,-0.006030,-0.006239,-0.006436,0.250971,0.250757,0.250704,0.250650,0.878602,0.877920,0.877224,0.876538
day_of_year,0.120640,0.131198,0.139138,0.153576,0.157418,-0.193835,-0.008993,0.023545,-0.033962,1.000000,0.134482,0.146565,0.160159,0.174348,0.141631,0.152880,0.166136,0.180762,0.150044,0.159433,0.170845,0.184102,0.163431,0.174959,0.187987,0.202064,0.150606,0.147979,0.144710,0.141949,-0.192670,-0.195117,-

In [225]:
def best_k_model(x_train, x_test, y_train, y_test, kmax):
    best_score = 0
    best_k = 1
    for k in range(1, kmax):
        knn_model = KNeighborsRegressor(n_neighbors=k).fit(x_train, y_train)
        score_knn = knn_model.score(x_test, y_test)
        if best_score < score_knn:
            best_score = score_knn
            best_k = k
    return KNeighborsRegressor(n_neighbors=best_k).fit(x_train, y_train), best_score, best_k


normalized = with_previous
X = normalized[['High-1', 'High-2', 'High-3', 'High-4',
                'Low-1', 'Low-2', 'Low-3', 'Low-4',
                'WetBulb_High-1', 'WetBulb_High-2', 'WetBulb_High-3', 'WetBulb_High-4',
                'WetBulb_Low-1', 'WetBulb_Low-2', 'WetBulb_Low-3', 'WetBulb_Low-4',
                'Humidity-1', 'Humidity-2', 'Humidity-3', 'Humidity-4',
                'WindSpeed-1', 'WindSpeed-2', 'WindSpeed-3', 'WindSpeed-4',
                'Pressure-1', 'Pressure-2', 'Pressure-3', 'Pressure-4']]
y = normalized['High']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

high_model, high_best_score, high_best_k = best_k_model(X_train, X_test, y_train, y_test, 20)

y = normalized['Low']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
low_model, low_best_score, low_best_k = best_k_model(X_train, X_test, y_train, y_test, 20)

y = normalized['Pressure']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
pressure_model, pressure_best_score, pressure_best_k = best_k_model(X_train, X_test, y_train, y_test, 20)

y = normalized['Humidity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
humidity_model, humidity_best_score, humidity_best_k = best_k_model(X_train, X_test, y_train, y_test, 20)

y = normalized['WindSpeed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
wind_model, wind_best_score, wind_best_k = best_k_model(X_train, X_test, y_train, y_test, 20)

print(f"High model: k:{high_best_k}, score:{high_best_score}\n")
print(f"Low model: k:{low_best_k}, score:{low_best_score}\n")
print(f"Pressure model: k:{pressure_best_k}, score:{pressure_best_score}\n")
print(f"Humidity model: k:{humidity_best_k}, score:{humidity_best_score}\n")
print(f"Wind Speed model: k:{wind_best_k}, score:{wind_best_score}")

High model: k:5, score:0.8134036202520939

Low model: k:6, score:0.8544821621362191

Pressure model: k:7, score:0.9587155878374853

Humidity model: k:5, score:0.47157986396555407

Wind Speed model: k:7, score:0.40808175091603516
